__Note__: The code used here was heavily inspired by the AI for Medicine Specialization Course 1 Week three programming assignments: [link](https://www.coursera.org/learn/ai-for-medical-diagnosis)

In [1]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import json
import os
from pprint import pprint
from collections import OrderedDict
from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from sys import getsizeof
from tqdm import tqdm

### Reading dataset description

In [2]:
file = open('../datasets_descriptions_json/task05_prostate_dataset.json')
data_description = json.load(file)

In [3]:
print(json.dumps(data_description, indent=4))

{
    "name": "PROSTATE",
    "description": "Prostate transitional zone and peripheral zone segmentation",
    "reference": "Radboud University, Nijmegen Medical Centre",
    "licence": "CC-BY-SA 4.0",
    "relase": "1.0 04/05/2018",
    "tensorImageSize": "4D",
    "quantitative": [
        0,
        1
    ],
    "modality": {
        "0": "T2",
        "1": "ADC"
    },
    "labels": {
        "0": "background",
        "1": "PZ",
        "2": "TZ"
    },
    "numTraining": 32,
    "numTest": 16,
    "training": [
        {
            "image": "./imagesTr/prostate_16.nii.gz",
            "label": "./labelsTr/prostate_16.nii.gz"
        },
        {
            "image": "./imagesTr/prostate_04.nii.gz",
            "label": "./labelsTr/prostate_04.nii.gz"
        },
        {
            "image": "./imagesTr/prostate_32.nii.gz",
            "label": "./labelsTr/prostate_32.nii.gz"
        },
        {
            "image": "./imagesTr/prostate_20.nii.gz",
            "label": "./labe

### Reading and visualizing prostate_00 nifti MRI and mask files

#### MRI file

In [4]:
mri_1 = nib.load("../../datasets/Task05_Prostate/train/images/prostate_00.nii.gz")

In [5]:
type(mri_1)

nibabel.nifti1.Nifti1Image

In [6]:
# Read Nifi Image object as numpy file
mri_1 = mri_1.get_fdata()

In [7]:
type(mri_1)

numpy.ndarray

In [8]:
mri_1.dtype

dtype('float64')

In [9]:
mri_1.shape

(320, 320, 15, 2)

In [10]:
# Range of voxel values (maximum value - minimum value)
print(mri_1[:, :, :, 0].ptp()) # Channel 0
print(mri_1[:, :, :, 1].ptp()) # Channel 1

1486.0
3619.0


In [11]:
getsizeof(mri_1) # 24.576144 MBs

24576144

In [12]:
getsizeof(mri_1.astype(np.float32)) # 12.288144 MBs

12288144

In [13]:
getsizeof(mri_1.astype(np.float16)) # 6.144144 MBs

6144144

#### Mask file

In [14]:
mri_1_mask = nib.load("../../datasets/Task05_Prostate/train/masks/prostate_00.nii.gz")

In [15]:
mri_1_mask = mri_1_mask.get_fdata()

In [16]:
mri_1_mask.dtype

dtype('float64')

In [17]:
np.unique(mri_1_mask)

array([0., 1., 2.])

In [18]:
getsizeof(mri_1_mask) # 12.288128 MBs

12288128

In [19]:
getsizeof(mri_1_mask.astype(np.float32)) # 6.144128 MBs

6144128

In [20]:
getsizeof(mri_1_mask.astype(np.uint8)) # 1.536128 MBs

1536128

In [21]:
mri_1_mask = mri_1_mask.astype(np.float32)

In [22]:
mri_1_mask.shape

(320, 320, 15)

### Visualizing nifti MRI file and Mask file

In [23]:
classes_dict = {
    'Background': 0,
    'Peripheral-Zone': 1,
    'Transition-Zone': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background', 'Peripheral-Zone', 'Transition-Zone', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_1.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_1[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_1_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_1_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 1))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Reading and visualizing prostate_01 nifti MRI and mask files

#### MRI file

In [24]:
mri_2 = nib.load("../../datasets/Task05_Prostate/train/images/prostate_01.nii.gz")

In [25]:
type(mri_2)

nibabel.nifti1.Nifti1Image

In [26]:
# Read Nifi Image object as numpy file
mri_2 = mri_2.get_fdata()

In [27]:
type(mri_2)

numpy.ndarray

In [28]:
mri_2.dtype

dtype('float64')

In [29]:
mri_2.shape

(320, 320, 20, 2)

In [30]:
# Range of voxel values (maximum value - minimum value)
print(mri_2[:, :, :, 0].ptp()) # Channel 0
print(mri_2[:, :, :, 1].ptp()) # Channel 1

1693.0
2948.0


#### Mask file

In [31]:
mri_2_mask = nib.load("../../datasets/Task05_Prostate/train/masks/prostate_01.nii.gz")

In [32]:
mri_2_mask = mri_2_mask.get_fdata()

In [33]:
mri_2_mask.dtype

dtype('float64')

In [34]:
np.unique(mri_2_mask)

array([0., 1., 2.])

In [35]:
mri_2_mask = mri_2_mask.astype(np.float32)

In [36]:
mri_2_mask.shape

(320, 320, 20)

### Visualizing nifti MRI file and Mask file

In [37]:
classes_dict = {
    'Background': 0,
    'Peripheral-Zone': 1,
    'Transition-Zone': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Peripheral-Zone', 'Transition-Zone', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_2.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_2[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_2_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_2_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 1))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Reading and visualizing prostate_02 nifti MRI and mask files

#### MRI file

In [38]:
mri_3 = nib.load("../../datasets/Task05_Prostate/train/images/prostate_02.nii.gz")

In [39]:
type(mri_3)

nibabel.nifti1.Nifti1Image

In [40]:
# Read Nifi Image object as numpy file
mri_3 = mri_3.get_fdata()

In [41]:
type(mri_3)

numpy.ndarray

In [42]:
mri_3.dtype

dtype('float64')

In [43]:
mri_3.shape

(320, 320, 24, 2)

In [44]:
# Range of voxel values (maximum value - minimum value)
print(mri_3[:, :, :, 0].ptp()) # Channel 0
print(mri_3[:, :, :, 1].ptp()) # Channel 1

2272.0
2871.0


#### Mask file

In [45]:
mri_3_mask = nib.load("../../datasets/Task05_Prostate/train/masks/prostate_02.nii.gz")

In [46]:
mri_3_mask = mri_3_mask.get_fdata()

In [47]:
mri_3_mask.dtype

dtype('float64')

In [48]:
np.unique(mri_3_mask)

array([0., 1., 2.])

In [49]:
mri_3_mask = mri_3_mask.astype(np.float32)

In [50]:
mri_3_mask.shape

(320, 320, 24)

### Visualizing nifti MRI file and Mask file

In [51]:
classes_dict = {
    'Background': 0,
    'Peripheral-Zone': 1,
    'Transition-Zone': 2
}

# Create button values
select_class = ToggleButtons(
    options=['Background','Peripheral-Zone', 'Transition-Zone', 'All'],
    description='Select Class:',
    disabled=False,
    button_style='info', 
    
)
# Create layer slider
select_layer = IntSlider(min=0, max=mri_3.shape[2] - 1, description='Select Layer', continuous_update=False)

    
# Define a function for plotting images
def plot(seg_class, layer, channel):
    print(f"Plotting Layer: {layer} | Label: {seg_class} | Channel: {channel}")
    fig = plt.figure(figsize=(20, 10))
    
    fig.add_subplot(1, 2, 1)
    plt.imshow(mri_3[:, :, layer, channel], cmap='gray');
    plt.title("MRI", fontsize=20)
    plt.axis('off')
    
    fig.add_subplot(1, 2, 2)
    if seg_class == "All":
        mask = mri_3_mask[:, :, layer]
        plt.title("Mask", fontsize=20)
        plt.imshow(mask)
        plt.axis('off');
    else:
        img_label = classes_dict[seg_class]
        mask = np.where(mri_3_mask[:, :, layer] == img_label, 255, 0)
        plt.title("Mask", fontsize=20)
        plt.imshow(mask, cmap='gray')
        plt.axis('off');

# Set channel to view:
#  Channel 0: T2
#  Channel 1: ADC    
    
# Use the interactive() tool to create the visualization
interactive(plot, seg_class=select_class, layer=select_layer, channel=(0, 1))

interactive(children=(ToggleButtons(button_style='info', description='Select Class:', options=('Background', '…

### Calculating MRI shape counter dictionary

In [52]:
data_dir = "../../datasets/Task05_Prostate/"

train_mri_paths = [os.path.join(data_dir + "train/images", x) for x in os.listdir(data_dir + "train/images")]
val_mri_paths = [os.path.join(data_dir + "val/images", x) for x in os.listdir(data_dir + "val/images")]
test_mri_paths = [os.path.join(data_dir + "test_images_for_model_prediction_submission", x) for x in os.listdir(data_dir + "test_images_for_model_prediction_submission")]

all_paths = train_mri_paths + val_mri_paths + test_mri_paths  # concatenate list of paths

shape_count_dict = {}

for path in tqdm(all_paths):
    mri = nib.load(path).get_fdata()
    shape = mri.shape
    
    if shape not in shape_count_dict.keys():
        shape_count_dict[shape] = 1
    else:
        shape_count_dict[shape] += 1   

100%|██████████| 48/48 [00:02<00:00, 16.61it/s]


In [53]:
shape_count_dict = OrderedDict(sorted(shape_count_dict.items(), key=lambda item: item[1], reverse=True))
pprint(shape_count_dict)

OrderedDict([((320, 320, 20, 2), 32),
             ((320, 320, 15, 2), 8),
             ((256, 256, 18, 2), 2),
             ((384, 384, 11, 2), 1),
             ((320, 320, 24, 2), 1),
             ((320, 320, 19, 2), 1),
             ((320, 320, 22, 2), 1),
             ((256, 256, 15, 2), 1),
             ((320, 320, 17, 2), 1)])


In [54]:
list_layers = []
for key, val in shape_count_dict.items():
    list_layers.append(key[2])

layers = np.array(list_layers)
print(layers.mean())
print(np.median(layers))
print(layers.min())
print(layers.max())

17.88888888888889
18.0
11
24


#### Checking which files do not have height and width equal to 320x320

In [55]:
for path in tqdm(all_paths):
    mri = nib.load(path).get_fdata()
    shape = mri.shape
    
    if shape[0] != 320:
        print(f"{path} {shape}") 

  4%|▍         | 2/48 [00:00<00:02, 16.40it/s]

../../datasets/Task05_Prostate/train/images/prostate_18.nii.gz (256, 256, 18, 2)


 10%|█         | 5/48 [00:00<00:02, 18.92it/s]

../../datasets/Task05_Prostate/train/images/prostate_28.nii.gz (384, 384, 11, 2)


 54%|█████▍    | 26/48 [00:01<00:01, 16.57it/s]

../../datasets/Task05_Prostate/val/images/prostate_37.nii.gz (256, 256, 15, 2)
../../datasets/Task05_Prostate/val/images/prostate_41.nii.gz (256, 256, 18, 2)


100%|██████████| 48/48 [00:02<00:00, 16.44it/s]
